# AG Grid Examples


In [ ]:
import pandas as pd

import nbappinator as nb
from nbappinator import ColMd, create_grid

In [ ]:
# Sample data
df = pd.DataFrame(
    {
        "name": ["Apple", "Microsoft", "Google", "Amazon", "Meta"],
        "price": [178.50, 378.91, 141.80, 178.25, 505.95],
        "change_pct": [0.0234, -0.0156, 0.0089, 0.0312, -0.0045],
        "volume": [52_340_000, 18_920_000, 21_450_000, 45_670_000, 12_340_000],
        "market_cap": [2.8e12, 2.9e12, 1.8e12, 1.9e12, 1.3e12],
    }
)

## 1. Standalone Grid

Use `create_grid()` to create a grid widget directly. This is useful when you just want a grid without the full App infrastructure.

In [ ]:
# Basic grid - just display the data
grid = create_grid(df, height=250)
grid

In [ ]:
# Grid with column formatting
columns = [
    ColMd(name="name", label="Company", pinned=True),
    ColMd(name="price", label="Price", format="dec", precision=2),
    ColMd(name="change_pct", label="Change %", format="perc", precision=2),
    ColMd(name="volume", label="Volume", format="mag_si", precision=1),
    ColMd(name="market_cap", label="Market Cap", format="mag_si", precision=2),
]

grid_formatted = create_grid(
    df,
    col_md=columns,
    height=250,
)
grid_formatted

In [ ]:
# Grid with click handler
def on_click(event):
    print(f"Clicked: {event.get('col_clicked')} = {event.get('value_clicked')}")
    print(f"Row data: {event.get('data')}")


grid_clickable = create_grid(
    df,
    col_md=columns,
    action=on_click,
    height=250,
)
grid_clickable

In [ ]:
# Access current selection
print("Current selection:", grid_clickable.current_selection)

## 2. Custom Grid Options

Pass any AG Grid options via the `grid_options` parameter:

In [ ]:
# Grid with custom AG Grid options
grid_custom = create_grid(
    df,
    col_md=columns,
    height=250,
    grid_options={
        "pagination": True,
        "paginationPageSize": 3,
        "suppressRowClickSelection": True,
        "enableCellTextSelection": True,
    },
)
grid_custom

## 3. Specifying AG Grid Version

By default, the latest AG Grid version is loaded from CDN. You can pin to a specific version:

In [ ]:
# Use a specific AG Grid version
grid_versioned = create_grid(
    df,
    col_md=columns,
    height=250,
    aggrid_version="35.1.0",  # Pin to specific version
)
grid_versioned

## 4. Width and Column Sizing

Control grid width and how columns are sized:

In [ ]:
# Grid with fixed width (smaller than default)
grid_small = create_grid(
    df,
    col_md=columns,
    height=250,
    width="600px",  # Fixed width instead of 100%
    size_columns_to_fit=False,
)
grid_small

In [ ]:
# Grid with columns sized to fit the grid width
grid_fit = create_grid(
    df,
    col_md=columns,
    height=250,
    size_columns_to_fit=True,  # Columns expand to fill grid width
)
grid_fit

In [ ]:
# Grid with auto-sizing disabled (columns use fixed widths)
# Note: auto_size_columns=True is the default
grid_no_autosize = create_grid(
    df,
    col_md=columns,
    height=250,
    auto_size_columns=False,  # Disable auto-sizing, use fixed column widths
)
grid_no_autosize

## 5. Grid within App

Use `app.dataframe()` to add a grid to an App.

In [ ]:
def update_grid(app):
    with app.messages:
        app.status("Updating grid...")

        precision = app["precision"]

        columns = [
            ColMd(name="name", label="Company", pinned=True),
            ColMd(name="price", label="Price", format="dec", precision=precision),
            ColMd(name="change_pct", label="Change %", format="perc", precision=precision),
            ColMd(name="volume", label="Volume", format="mag_si", precision=1),
            ColMd(name="market_cap", label="Market Cap", format="mag_si", precision=2),
        ]

        app.tab(0).clear().dataframe(
            "grid",
            df,
            columns=columns,
            on_click=lambda x: print(f"Clicked: {x}"),
            height=300,
        )

        app.done("Grid updated")


app = nb.App(tabs=["Grid"], footer="Messages", header="Config")

app.config.select("precision", options=[0, 1, 2, 3, 4], default=2, label="Precision")
app.config.button("update", on_click=update_grid, label="Update Grid", status=True)

update_grid(app)

app.display()

## 6. Tree Data (Enterprise Feature)

Tree Data requires AG Grid Enterprise. Without enterprise mode, the data will be shown as a flat table with the path column visible. See `8_aggrid_enterprise.ipynb` for enterprise examples with working tree data.

In [ ]:
tree_df = pd.DataFrame(
    {
        "path": [
            "Tech",
            "Tech/Hardware",
            "Tech/Hardware/Apple",
            "Tech/Hardware/Dell",
            "Tech/Software",
            "Tech/Software/Microsoft",
            "Tech/Software/Oracle",
            "Finance",
            "Finance/Banks",
            "Finance/Banks/JPMorgan",
            "Finance/Banks/BofA",
        ],
        "revenue": [500e9, 200e9, 150e9, 50e9, 300e9, 200e9, 100e9, 400e9, 250e9, 150e9, 100e9],
        "employees": [
            1_500_000,
            500_000,
            150_000,
            130_000,
            800_000,
            220_000,
            140_000,
            900_000,
            500_000,
            290_000,
            210_000,
        ],
    }
)

tree_columns = [
    ColMd(name="revenue", label="Revenue", format="mag_si", precision=1),
    ColMd(name="employees", label="Employees", format="mag_si", precision=1),
]

# Community mode - will show warning that tree data requires enterprise
tree_grid = create_grid(
    tree_df,
    is_tree=True,
    pathcol="path",
    pathdelim="/",
    col_md=tree_columns,
    height=350,
)
tree_grid